# Forest Cover - Neural Network

Import libraries

In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.optimizers import RMSprop, Adam, Nadam, Adamax
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Import Data

In [27]:
data = pd.read_csv('train.csv')

In [28]:
data.head(5)

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [29]:
data = data.drop('Id', axis=1)
data['Cover_Type'] = data['Cover_Type'].astype('category')

In [30]:
data.head(5)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [31]:
data.dtypes

Elevation                                int64
Aspect                                   int64
Slope                                    int64
Horizontal_Distance_To_Hydrology         int64
Vertical_Distance_To_Hydrology           int64
Horizontal_Distance_To_Roadways          int64
Hillshade_9am                            int64
Hillshade_Noon                           int64
Hillshade_3pm                            int64
Horizontal_Distance_To_Fire_Points       int64
Wilderness_Area1                         int64
Wilderness_Area2                         int64
Wilderness_Area3                         int64
Wilderness_Area4                         int64
Soil_Type1                               int64
Soil_Type2                               int64
Soil_Type3                               int64
Soil_Type4                               int64
Soil_Type5                               int64
Soil_Type6                               int64
Soil_Type7                               int64
Soil_Type8   

In [32]:
features = data.drop('Cover_Type', axis=1)
features.head(5)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,0
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,0
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,0
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,0
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,0


In [33]:
types = data['Cover_Type']
types.head(5)

0    5
1    5
2    2
3    2
4    5
Name: Cover_Type, dtype: category
Categories (7, int64): [1, 2, 3, 4, 5, 6, 7]

### Train/Test Split

In [34]:
train_features, test_features, train_types, test_types = train_test_split(features, types, test_size = 0.15, random_state = 42)

## Neural Network

In [40]:
def NNmodel():
    myInputs = Input(shape=(54,))

    x = Dense(108)(myInputs)
    x = Activation('sigmoid')(x)
    x = Dropout(0.26)(x)

    x = Dense(162)(x)
    x = Activation('sigmoid')(x)
    x = Dropout(0.39)(x)

    x = Dense(54)(myInputs)
    x = Activation('sigmoid')(x)
    x = Dropout(0.13)(x)
    
    x = Dense(51)(myInputs)
    x = Activation('sigmoid')(x)
    x = Dropout(0.12)(x)
    
    x = Dense(12)(myInputs)
    x = Activation('sigmoid')(x)
    x = Dropout(0.03)(x)
    
    x = Dense(10)(myInputs)
    x = Activation('sigmoid')(x)
    x = Dropout(0.03)(x)
    
    x = Dense(8)(myInputs)
    x = Activation('sigmoid')(x)
    x = Dropout(0.02)(x)

    myPredicts = Dense(1)(x)
    myModel = Model(inputs=myInputs, outputs=myPredicts)
    
    return myModel

In [41]:
model = NNmodel()
# myRMS = RMSprop(lr=0.0005, rho=0.9, epsilon=None, decay=0.0)
model.compile(optimizer=myRMS, loss='mse', metrics=['mse'])

scaler1 = StandardScaler()
scaler1 = scaler1.fit(train_features)
xScaled0 = scaler1.transform(train_features)

scaler2 = MinMaxScaler(feature_range=(0,1))
scaler2 = scaler2.fit(xScaled0)
xScaled = scaler2.transform(xScaled0)

model.fit(xScaled, train_types, epochs=25, verbose=0)

xTestScale0 = scaler1.transform(test_features)
xTestScale = scaler2.transform(xTestScale0)

y_pred = model.predict(xTestScale)

In [43]:
model = Sequential()

model.add(Dense(54,
				input_dim=54,
				kernel_initializer='uniform',
				activation='relu'))
model.add(Dropout(0.125))

model.add(Dense(108,
				kernel_initializer='uniform',
				activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(162,
				kernel_initializer='uniform',
				activation='relu'))
model.add(Dropout(0.375))

model.add(Dense(108,
				kernel_initializer='uniform',
				activation='sigmoid'))
model.add(Dropout(0.25))



In [44]:
types.max


<bound method Series.max of 0        5
1        5
2        2
3        2
4        5
5        2
6        5
7        5
8        5
9        5
10       5
11       2
12       2
13       5
14       5
15       5
16       5
17       5
18       5
19       5
20       5
21       2
22       5
23       5
24       5
25       5
26       5
27       2
28       2
29       5
        ..
15090    3
15091    3
15092    3
15093    6
15094    6
15095    6
15096    6
15097    3
15098    3
15099    6
15100    3
15101    6
15102    3
15103    6
15104    3
15105    6
15106    3
15107    3
15108    3
15109    6
15110    6
15111    6
15112    6
15113    3
15114    3
15115    3
15116    3
15117    3
15118    3
15119    3
Name: Cover_Type, Length: 15120, dtype: category
Categories (7, int64): [1, 2, 3, 4, 5, 6, 7]>